# XML Parser

**15-688 STUDENTS only**

In this problem, you're going to write a function that will parse (a simplified version of) XML files into a Python object. Although it's not recommended that you use the parser you construct for anything serious (many excellent Python libraries already exist for parsing XML, such as the lxml library), XML files represent a fairly complex file format, that necessitates using regular expressions and recursion (or a stack) to parse these in a reasonably efficient manner.  So while it's not likely that you will need to write you own XML parser, chances are if/when you _do_ need to write a parser for some format for which there exists no good Python library, the techniques you use here will be useful for writing this parser as well.

XML stands for the eXtensible Markup Language.  It appeared as a successor to SGML (standard generalized markup language) and HTML (hypertext markup language, the standard for displaying web pages), but with some additional structure that makes the documents more well-defined; for instance, in HTML it's common for open tags to appear without a corresponding close tag, which is not allowed in pure XML.

You may already be familiar with XML, but if not the official resource for learning about the format is here https://www.w3.org/XML/ and a good resource with some concrete examples is here http://www.w3schools.com/xml/.  We'll assume here that you're broadly familiar with the basic ideas behind XML, and just describe what you need to know to complete the parser for this assignment.

Here is an example XML document:
    
    <?xml version="1.0" encoding="UTF-8"?>
    <!-- This is a comment -->
    <note date="8/31/12">
        <to>Tove</to>
        <from>Jani</from>
        <heading type="Reminder"/>
        <body>Don't forget me this weekend!</body>
        <!-- This is a multiline comment,
             which take a bit of care to parse -->
    </note>
    
There are a few elements here of importance.
1. Tags are denoted `<tag_name>content</tag_name>` where `<tag_name>` is the opening tag and `</tag_name>` is the closing tag.  All text (including whitespace, and subtags, etc) between these tags is the content.
2. Attributes follow a tag, and are written as a list of `attr_name="attribute_value"` pairs, where we can use either double quotes or single quotes around the attribute value.  If you use double quotes then a single quote can appear in the text and vice versa.  There can be whitespace around the equals sign or not.
3. If a tag has no content `<tag_name attr_name="attribute_value"></tag_name>` can be abbreviated as the open/close tag `<tag_name attr_name="attribute_value"/>`. In some cases, such as in HTML5, you might come across tags that have no content but aren't closed, such as `<meta ...>` and `<link ...>` tags. **However, as we are dealing with XML (a stricter context), tags are required to be closed.** If you're interested, [this document](https://www.w3schools.com/html/html_xhtml.asp) on XHTML vs HTML touches upon this idea.
4. A XML prologue is written as `<?tag_name attr_name="attribute_value"?>`.  It has no close tag.  We'll also consider documents that allow for an HTML declaration, such as `<!DOCTYPE html>` (this will let us parse some HTML documents that are well-formed enough to also parse as valid XML).
5. Comments are denoted by `<!-- comment_text -->`  and the comment text can span multiple lines.

## Q1: Regular expression for identifying tags

First, we'll use regular expressions to identify tags and other elements of XML files.  Specifically, you'll need to create 6 regular expressions that locate open tags, close tags, open/close tags, comments, xml_prolog, and html declarations.  For the open, close and open/close tags, make sure that your regular expression also matches and returns 1) the tag name, and 2) all the attributes (i.e., they should contain two groups, one for each of these two components). 

Note that in order to match the solutions, your `tag_open` regex should _both_ open _and_ open/close tags (it's posssible, but substantially trickier, to create regex that matches _only_ open and _not open/close tags, so we'll handle that logic separate).  Comments may be split across multiple lines, but you can assume that all other tags must occur on a single line (without newlines within the tag itself).

Fill in the in the following regular expressions, then you can use the `mugrade` tests with the provided function.

In [ ]:
import re
import requests
import mugrade

tag_open = re.compile("")
tag_close = re.compile("")
tag_open_close = re.compile("")

comment = re.compile("")
xml_prolog = re.compile("")
html_declaration = re.compile("")

@mugrade.local_tests
def tag_regex(inp):
    return {
        "tag_open": tag_open.findall(inp),
        "tag_close": tag_close.findall(inp),
        "tag_open_close": tag_open_close.findall(inp),
        "comment": comment.findall(inp),
        "xml_prolog": xml_prolog.findall(inp),
        "html_declaration": html_declaration.findall(inp)
    }

## Q2: XML Parser

Using the regular expressions above, now you'll write an XML parser (although technically you don't _have_ to use them, you could try to write a complete XML parser using a single regular extended expression if you really want to, but we would highly advise against this).  Specifically, you should fill in the `__init__` function for the class prototype below (more discussion of the actual implemeentation follows below).

We've provided you with some test cases to make sure that your parser works correctly.

In [ ]:
class XMLNode:
    def __init__(self, tag, attributes, content):
        self.tag = tag # The tag <tag>
        self.attributes = attributes # A dictionary from attributes to values
        self.children = [] # A list of either XMLNode objects and strings, corresponding to tags and text between them.
        self.content = content # A string of everything in the original document inside these tags

        pass
    
@mugrade.local_tests
def create_xml_tree(text):
    """parse an XML tree from a string"""
    return XMLNode("", {}, text)

There is a lot that this function needs to do, which is best explained by an example.  We'll eventually parse the `mugrade.mugrade_test_cases.test_xml` above using the command:
    
    root = XMLNode("", {}, mugrade.mugrade_test_cases.test_xml)

This will create a root node with an empty tag, an empty dictionary for the attributes, and created by parsing the test_snippet; the `content` attribute here will contain the entire test snippet.

The `children` attribute is a list that contains a single XMLNode instance, corresponding to the `note` tag.  This instance is created (you don't call this function directly, rather each node must recursively create all its children, so the call above would recursively make the following call to create its child) by calling `XMLNode` with the parameters

    XMLNode("note", {"date":"8/31/12"}, mugrade.mugrade_test_cases.test_xml[87:])
 
(`mugrade.mugrade_test_cases.test_xml[87:]` just happens to be the position that immediately follows the `<note>` open tag).  This child node will have the given tag and attributes, plus content given by only the `content` equals to the string that occurs _within_ the note open and close tags.  It will similarly have four children, one corresponding to each of the subtags.  In addition to the provided test cases, the following code illustrates how you can use `XMLNode` to parse test_snippet, and what the structure is after you perform the parsing.

In [ ]:
import test_hw1_xml_parser
root = XMLNode("", {}, test_hw1_xml_parser.test_xml)

print("root.tag: ", root.tag)
print("root.attributes: ", root.attributes)
print("root.content: ", repr(root.content))
print("root.children: ", root.children)
print("")
print("note.tag: ", root.children[0].tag)
print("note.attributes: ", root.children[0].attributes)
print("note.content: ", repr(root.children[0].content))
print("note.children: ", root.children[0].children)
print("")
print("to.tag: ", root.children[0].children[0].tag)
print("to.attributes: ", root.children[0].children[0].attributes)
print("to.content: ", repr(root.children[0].children[0].content))
print("to.children: ", root.children[0].children[0].children)
print("")
print("heading.tag: ", root.children[0].children[2].tag)
print("heading.attributes: ", root.children[0].children[2].attributes)
print("heading.content: ", repr(root.children[0].children[2].content))
print("heading.children: ", root.children[0].children[2].children)

If written correctly, this should produce the following output
```python
root.tag:  
root.attributes:  {}
root.content:  '<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE xml> <!-- not actually valid xml-->\n<!-- This is a comment -->\n<note date="8/31/12">\n    <to>Tove</to>\n    <from>Jani</from>\n    <heading type="Reminder"/>\n    <body>Don\'t forget me this weekend!</body>\n    <!-- This is a multiline comment,\n         which take a bit of care to parse -->\n</note>\n'
root.children:  [<__main__.XMLNode instance at 0x10425c0e0>]

note.tag:  note
note.attributes:  {'date': '8/31/12'}
note.content:  '\n    <to>Tove</to>\n    <from>Jani</from>\n    <heading type="Reminder"/>\n    <body>Don\'t forget me this weekend!</body>\n    <!-- This is a multiline comment,\n         which take a bit of care to parse -->\n'
note.children:  [<__main__.XMLNode instance at 0x10425c098>, <__main__.XMLNode instance at 0x10425c128>, <__main__.XMLNode instance at 0x10425c368>, <__main__.XMLNode instance at 0x10425c488>]

to.tag:  to
to.attributes:  {}
to.content:  'Tove'
to.children:  []

heading.tag:  heading
heading.attributes:  {'type': 'Reminder'}
heading.content:  ''
heading.children:  []
```

To verify the rest of the implementation, be sure to pass all the local test cases.

Lets discuss in a bit more detail how the XML parsing will work algorithmically.  We begin the initializer by copying the provided parameters to the class attributes.  Note that if you want you could make a full string copy here, but we don't bother.  Now we begin parsing the file, which we do by repeating the following logic until termination:
1. Look for the next xml tag (or comment, etc), in the file.  This is best done by finding the next `'<'` character.  If you can't find any, return.
2. If it's an xml prolog, html declaration, or comment, ignore this portion, and continue parsing after the prolog, declaration, or comment (i.e., throw away whatever information is contained in these portions)
3. If it's an open tag, read its tag and attributes (you'll likely want to use a regular expression to parse the attributes as well, but we leave this up to you).  If it's just an open tag, then recursively create an XMLNode object initializer this tag and attributes, and the content that occurred after the open tag.  If it's an open/close tag, create a XMLNode the same as before but with empty content.
4. If it's a close tag, make sure that the close tag matches the tag originally provided to the current XMLNode constructor (otherwise, we have a situation where one tag is closed with a different tag), and raise an Exception if not.  If the tags do match, then truncate the content to contain only the content before the closed tag matched, and return.

Some hints that we believe will be helpful:
1. Keep track of the current position where you are parsing the file, and make sure to properly increment this so you move past any tag that you have parsed.
2. Make use of the `match = regular_expression_obj.match(string, pos)` function, which looks for a match to the regular expression starting _exactly_ as position `pos` in `string`.  If this function returns `None`, then the regular expression did not match.  In the returned `match` object, `match.end()` contains the position where the match ended.

## Q3: Searching for tags

One of the nicer elements of the `BeautifulSoup` library is the ability to quickly search for tags that have certain attributes, without worrying about the specific structure of the model (i.e., how many levels deep the tag is, how many may exist in the document etc).  We're going to implement a similar function in our `XMLNode` class

For those who haven't seen the `**kwargs` parameter before, this is just a way to pass a variable-length list of parameters to a Python function as function parameters.  For example, you could call `find` via

```python
root.find("link", rel="stylesheet") -> [XMLNode("link", {...}, "...")]
```

and in the `find` function, `kwargs` would be a dictionary equal to `{"rel":"stylesheet"}`.

This function should return a list of _all_ XMLNodes that are descendents (children, children of children, etc), of the node you call it on. Providing no filter should match all nodes.


In [239]:
# Fill out this function and move it into the XMLNode class above.
def find(self, tag, **kwargs):
    """
    Search for a given tag and atributes anywhere in the XML tree

    Args:
        tag (string): tag to match
        kwargs (dictionary): list of attribute name / attribute value pairs to match

    Returns:
        (list): a list of XMLNode objects that match from anywhere in the tree
    """
    pass

After you have created the class with this `find` function, you can run the following tests.

In [ ]:
@mugrade.local_tests
def create_searchable_xml_tree(text):
    """parse an XML tree from a string"""
    return XMLNode("", {}, text)